# PyMNtos 2021-11-08 Python 3.10: Structural Pattern Matching presentation demo.




In [1]:
import time
import pandas as pd

# Obligatory ~simplistic~ execution time test

In [2]:
def convert_field_elif(value: int, conversion: str):
    """Do some conversion on the resulting object."""
    if conversion is None:
        return value
    elif conversion == 's':
        return str(value)
    elif conversion == 'r':
        return repr(value)
    elif conversion == 'a':
        return ascii(value)
    else:
        raise ValueError(f"Unknown conversion specifier {conversion}")

In [3]:
def convert_field_match(value: int, conversion: str):
    """Do some conversion on the resulting object."""
    match conversion:
        case None:
            return value
        case 's':
            return str(value)
        case 'r':
            return repr(value)
        case 'a':
            return ascii(value)
        case _:
            raise ValueError(f"Unknown conversion specifier {conversion}")

In [21]:
def compare_elif_and_match_execution_time(loops: int):
    """Run the convert_field_* methods over a pre-defined loop while calculating exectuion time."""
    start = time.time()
    for i in range(0, loops + 1):
        [convert_field_elif(i, _conversion) for _conversion in [None, 's', 'r', 'a']]
    elif_time = time.time() - start

    start = time.time()
    for i in range(0, loops + 1):
        [convert_field_match(i, _conversion) for _conversion in [None, 's', 'r', 'a']]
    match_time = time.time() - start
    
    return elif_time,"->" if elif_time > match_time else "<-", match_time

results = []
for loop in [100, 1000, 1000000, 10000000]:
    results.append((loop, *compare_elif_and_match_execution_time(loop)))

pd.DataFrame(results, columns=['loop', 'elif_time seconds', 'faster', 'match_time seconds'])

,loop,elif_time seconds,faster,match_time seconds
0,100,0.000287,<-,0.000314
1,1000,0.003229,->,0.002997
2,1000000,0.635925,<-,0.636027
3,10000000,6.267407,<-,6.338647


### No real conclusive results in terms of execution time... 
### I'm sure folks who have more working interpreter knowledge would have some insights.

# or pattern, wildpattern, guards

In [20]:
def factorial(n: int):
    """Let's add another way to implement Factorial!"""
    match n:    
        case n if n > 1:
            return n * factorial(n-1)
        case 0 | 1 as m:
            print(f'from 0 or 1 {m}')
            return 1
        case n if any(n < 0, n <= -1):
            raise ValueError('n must be greater than one')
        case _:
            raise TypeError(
                f'Unhandled type: {type(n)}. Accepts int.')
        
for n in range(0, 9 + 1):
    print(f'{n}: {factorial(n)}')
factorial(-4)

from 0 or 1 0
0: 1
from 0 or 1 1
1: 1
from 0 or 1 1
2: 2
from 0 or 1 1
3: 6
from 0 or 1 1
4: 24
from 0 or 1 1
5: 120
from 0 or 1 1
6: 720
from 0 or 1 1
7: 5040
from 0 or 1 1
8: 40320
from 0 or 1 1
9: 362880


TypeError: any() takes exactly one argument (2 given)

In [6]:
factorial("tomato")

TypeError: '>' not supported between instances of 'str' and 'int'

In [7]:
class Coord:
    """A class to represent coordinate values."""
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [8]:
coord = Coord(1, 2)
match coord:
    case Coord(0, 0):
        print("at the origin")
    case Coord(0, y):
        print(f"On vertical axis, y = {y}")
    case Coord(x, 0):
        print(f"On horizontal axis, x = {x}")
    case Coord(x, y):
        print(f"at {coord}")
    case _:
        print("unhandled")

TypeError: Coord() accepts 0 positional sub-patterns (2 given)

## Why doesn't this work? We need to specify what is being matched to.

In [9]:
coord = Coord(1, 0)
match coord:
    case Coord(x=0, y=0):
        print("at the origin")
    case Coord(x=0, y=y):
        print(f"On vertical axis, y = {y}")
    case Coord(x=x, y=0):
        print(f"On horizontal axis, x = {x}")
    case Coord(x=x, y=y):
        print(f"at {coord}")
    case _:
        print("unhandled")

On horizontal axis, x = 1


## We can use the new `__match_args__` class field to specify arguments to match to for a class

In [10]:
class Coord:
    """A class to represent coordinate values."""
    
    __match_args__ = ("x", "y")
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [11]:
coord = Coord(0, 1)
match coord:
    case Coord(0, 0):
        print("at the origin")
    case Coord(0, y):
        print(f"On vertical axis, y = {y}")
    case Coord(x, 0):
        print(f"On horizontal axis, x = {x}")
    case Coord(x, y):
        print(f"at {coord}")
    case _:
        print("unhandled")

On vertical axis, y = 1


### Python does a great job with tuple unpacking. Let's take it one step further.

see: https://www.python.org/dev/peps/pep-0642

In [18]:
def color_unpacking(color: tuple):
    """Normalize to name, r, g, b, alpha."""
    match color:
        case (int(r), int(g), int(b)):
            name = "missing name"
            a = 0
        case (int(r), int(g), int(b), int(a)):
            name = "missing name"
        case (str(name), (int(r), int(g), int(b))):
            a = 0
        case (str(name), (int(r), int(g), int(b), int(a))):
            pass
        case _:
            raise ValueError("Unknown color info.")
    print(name, (r, g, b, a))

color_unpacking((255,255,255))
color_unpacking(('Purple', (101, 17, 107)))
color_unpacking(('Red', (255, 0, 0, 0)))
color_unpacking(("Green", (0, 255, "0")))

missing name (255, 255, 255, 0)
Purple (101, 17, 107, 0)
Red (255, 0, 0, 0)


ValueError: Unknown color info.

### This handles type validaiton as well.

In [14]:
color_unpacking((True, (255, 255, 255)))

ValueError: Unknown color info.

In [16]:
for (n, l, z) in (1,2,3):
    print(n,l,z)

TypeError: cannot unpack non-iterable int object